# Census Data API: ACS 2019 Statewide Population 

In [14]:
from urllib import request
import json

census_api_key= '75b228a93b0b0592dd6d820dc0077020a546514a'
url_str= 'https://api.census.gov/data/2019/acs/acs5?get=B01003_001E,NAME,GEO_ID&for=state:*&key='+census_api_key

predicates= {}
get_vars= ["NAME","GEO_ID","B01003_001E"]
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:*"
r= requests.get(url_str, params= predicates)

In [15]:
print(r.text)

[["B01003_001E","NAME","GEO_ID","state"],
["4876250","Alabama","0400000US01","01"],
["737068","Alaska","0400000US02","02"],
["7050299","Arizona","0400000US04","04"],
["2999370","Arkansas","0400000US05","05"],
["39283497","California","0400000US06","06"],
["5610349","Colorado","0400000US08","08"],
["957248","Delaware","0400000US10","10"],
["692683","District of Columbia","0400000US11","11"],
["3575074","Connecticut","0400000US09","09"],
["20901636","Florida","0400000US12","12"],
["10403847","Georgia","0400000US13","13"],
["1717750","Idaho","0400000US16","16"],
["1422094","Hawaii","0400000US15","15"],
["12770631","Illinois","0400000US17","17"],
["6665703","Indiana","0400000US18","18"],
["3139508","Iowa","0400000US19","19"],
["2910652","Kansas","0400000US20","20"],
["4449052","Kentucky","0400000US21","21"],
["4664362","Louisiana","0400000US22","22"],
["1335492","Maine","0400000US23","23"],
["6018848","Maryland","0400000US24","24"],
["6850553","Massachusetts","0400000US25","25"],
["9965265

In [16]:
print(r.json()[0])

['B01003_001E', 'NAME', 'GEO_ID', 'state']


In [17]:
col_names= ["total_pop","Name", "GEO_ID", "state"]

In [18]:
import pandas as pd
df= pd.DataFrame (columns=col_names, data=r.json()[1:])
#fix data types
df["total_pop"]= df["total_pop"].astype(int)
df.head()

,total_pop,Name,GEO_ID,state
0,4876250,Alabama,0400000US01,01
1,737068,Alaska,0400000US02,02
2,7050299,Arizona,0400000US04,04
3,2999370,Arkansas,0400000US05,05
4,39283497,California,0400000US06,06


In [65]:
df.to_csv('TEST.csv', encoding='utf-8', index=False)

# Census Data TARGET: Mapping